In [1]:
import pandas as pd

import aidd.sys.config as cfg
from aidd.utils.data_io import read_data, save_data
from aidd.utils.data_io import read_pickle, save_pickle

from aidd.modeling.evaluations import user_mape

In [2]:
# POLE_CNT 별로 스케일러와 최고 성능의 모델 불러오기
ms_data = {     # ms: modeling result data
    pckey: {
        'SCALER': read_pickle(f'DUMP,SCALER,{pckey}'),
        'MODEL': read_pickle(f'DUMP,MODELS,{pckey},BEST')
    } for pckey in cfg.DATA_PC_TYPE
}

In [3]:
# 전처리된 서비스 데이터 불러오기
df = read_data('ONLINE')

In [4]:
# 임시 코드: 서비스 데이터와 학습 데이터의 컬럼 순서 확인
_bdf = read_data('PP,SL', dtype={'CONS_ID': str})
_o_cols = df.columns.tolist()
_b_cols = _bdf.columns.tolist()
for idx in range(len(_o_cols)):
    if _o_cols[idx] != _b_cols[idx]:
        raise('컬럼이 일치하지 않음')

In [5]:
del _bdf, _o_cols, _b_cols

In [6]:
return_df = df[['CONS_ID', 'TOTAL_CONS_COST']].copy()
return_tcc = []     # tcc: total cons cost
# 모델에 사용한 컬럼정보 불러오기
training_cols = read_pickle(file_code='DUMP,MODELING_COLS')

In [7]:
# 임시 코드: 모델링 코드와 서비스 컬럼 데이터 다시 확인
_temp_cols = df.columns[5:].tolist()
for idx in range(len(_temp_cols)):
    if _temp_cols[idx] != training_cols[idx]:
        raise('컬럼이 일치하지 않음')
del _temp_cols

In [8]:
X = df[training_cols].copy()

In [9]:
for _, row in X.iterrows():
    row_df = pd.DataFrame(row).transpose()
    # 인덱스 초기화: 이 작업을 수행해야 각 df의 첫 번째 인덱스가 0이 됨
    # 0으로 해야 레코드의 POLE_CNT를 추출해 각각의 스케일러와 모델에 적용해 줄 수 있음
    row_df.reset_index(drop=True, inplace=True)
    pckey = row_df.loc[0, 'POLE_CNT'].astype(int)
    if pckey == 1:
        scaler = ms_data['1']['SCALER']
        model = ms_data['1']['MODEL']
    else:
        scaler = ms_data['N1']['SCALER']
        model = ms_data['N1']['MODEL']
    ascaler = ms_data['ALL']['SCALER']
    amodel = ms_data['ALL']['MODEL']
    
    s_data = scaler.transform(row_df)
    as_data = ascaler.transform(row_df)
    p = model.predict(s_data)[0]
    ap = amodel.predict(as_data)[0]
    return_tcc.append([p, ap])

/home/freeman/anaconda3/envs/t213p310/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
/home/freeman/anaconda3/envs/t213p310/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
/home/freeman/anaconda3/envs/t213p310/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but ElasticNet was fitted with feature names
  warnings.warn(
/home/freeman/anaconda3/envs/t213p310/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
/home/freeman/anaconda3/envs/t213p310/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but GradientBoost

In [10]:
return_tcc

[[11713656.003436096, 14171198.12148257],
 [4123721.775615843, 4277901.706869179],
 [10463101.806429254, 13254288.579833094]]

In [11]:
return_df

,CONS_ID,TOTAL_CONS_COST
0,477420204194,9076645
1,475920223725,2176378
2,474620226651,9512744


In [12]:
return_df.loc[:, ['ALL_TCC', 'PCKEY_TCC']] = return_tcc

In [13]:
return_df

,CONS_ID,TOTAL_CONS_COST,ALL_TCC,PCKEY_TCC
0,477420204194,9076645,1.171366e+07,1.417120e+07
1,475920223725,2176378,4.123722e+06,4.277902e+06
2,474620226651,9512744,1.046310e+07,1.325429e+07


In [15]:
for idx, row in return_df.iterrows():
    y = return_df.loc[idx, 'TOTAL_CONS_COST']
    p = return_df.loc[idx, 'ALL_TCC']
    ap = return_df.loc[idx, 'PCKEY_TCC']
    print(f' p: {y} vs {p} = {abs(y-p)/y*100:.3f}')
    print(f'ap: {y} vs {ap} = {abs(y-ap)/y*100:.3f}')

 p: 9076645 vs 11713656.003436096 = 29.053
ap: 9076645 vs 14171198.12148257 = 56.128
 p: 2176378 vs 4123721.775615843 = 89.476
ap: 2176378 vs 4277901.706869179 = 96.561
 p: 9512744 vs 10463101.806429254 = 9.990
ap: 9512744 vs 13254288.579833094 = 39.332
